## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros

model_name = "refined_traces_models"

# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=[model_name],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-16 09:53:41 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=61221
2024-12-16 09:53:41 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=61221
2024-12-16 09:53:41 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=61221
2024-12-16 09:53:41 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=61221
2024-12-16 09:53:42 [info     ] prepared 1 input batches.      filename=bydate.py lineno=92 process=61221


## Run the model

This automatically registers the model outputs as duckdb tables.

In [ ]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model=model_name,
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-16 09:53:42 [info     ] Executing model function...    filename=testutils.py lineno=220 process=61221
2024-12-16 09:53:42 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=61221
2024-12-16 09:53:43 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53 process=61221
2024-12-16 09:53:43 [info     ] duckdb dataset='ingestion/blocks_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=61221
2024-12-16 09:53:44 [info     ] registered view: 'ingestion_blocks_v1' using 1 parquet paths filename=client.py lineno=53 process=61221
2024-12-16 09:53:44 [info     ] duckdb dataset='ingestion/traces_v1' using 1/22 parquet paths, first path is gs:

## Verify model results - Traces

In [ ]:
duckdb_client.sql("SELECT * FROM daily_trace_calls_tr_to_v1 ORDER BY count_transactions_called DESC LIMIT 10")

### Test Transaction Metrics

In [ ]:
duckdb_client.sql("""
                  SELECT trace_to_address,
                    count_transactions_called_with_internal_type_call, count_transactions_called_with_internal_type_call_or_delegate,
                    count_transactions_called
                  FROM daily_trace_calls_tr_to_v1
                  ORDER BY count_transactions_called_with_internal_type_call DESC
                  LIMIT 10
                  """)

### Test Gas Used Metrics

In [ ]:
duckdb_client.sql("""
                  SELECT
                    trace_to_address,
                  
                    sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call,
                    sum_tx_l2_gas_used_amortized_by_call_tx_success_called_with_internal_type_call,
                  
                    sum_tx_l2_fee_native_minus_subtraces_tx_success_called_with_internal_type_call
                    sum_tx_l2_fee_native_amortized_by_call_tx_success_called_with_internal_type_call,
                    sum_tx_fee_native_amortized_by_call_tx_success_called_with_internal_type_call,
                  
                    count_transactions_called_with_internal_type_call,
                    count_transactions_called_with_internal_type_call_or_delegate,
                    count_transactions_called
                  
                  FROM daily_trace_calls_tr_to_v1
                  ORDER BY sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call DESC
                  LIMIT 10
                  """)

In [ ]:
duckdb_client.sql("""
    SELECT * FROM refined_traces_v1
                  where transaction_hash = '0xc620133c2339f36d8bfae889ea29e9986a70182f7bbe3380d0622f3801619eda'
                  AND block_number = 128145924
                  ORDER BY trace_address ASC
                  LIMIT 10
                  """)

### Check the data output size

In [ ]:
duckdb_client.sql("SELECT COUNT(*) AS interm_num_calls, COUNT(DISTINCT transaction_hash) AS num_txs, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM refined_traces_v1")

In [ ]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT transaction_hash) AS num_txs, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM aggregated_traces_tr_to_hash_v1")

In [ ]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM daily_trace_calls_tr_to_v1")

### Get table schema

In [ ]:
duckdb_client.sql("DESCRIBE daily_trace_calls_tr_to_v1")
